In [57]:
import pandas as pd
import numpy as np

from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.feature_extraction import DataSource
from pals.PLAGE import PLAGE
from pals.ORA import ORA
from pals.GSEA import GSEA
from pals.common import *

from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests



In [58]:
mz_df = pd.DataFrame(columns = ['m/z', 'retention_time'])

mz_df['m/z'] = [np.random.randint(1, 150) for i in range(7375)]
mz_df['retention_time'] = [np.random.randint(1, 300) for i in range(7375)]
print(mz_df)


      m/z  retention_time
0      74             179
1     108             218
2      14             136
3      89             156
4     147              59
...   ...             ...
7370   93              80
7371   91              99
7372   10             206
7373   11             169
7374   42              78

[7375 rows x 2 columns]


In [59]:
intdf, annodf, groups = load_data(".\\int_df.csv", ".\\annotation_df.csv") #path to beer data 
groups

FileNotFoundError: [Errno 2] No such file or directory: '.\\int_df.csv'

In [49]:
comparisons = [
    ('beer1', 'beer2'), 
    # ('beer3', 'beer4')
]

In [50]:
experimental_design = {
    'groups': groups,
    'comparisons': []
}
for case, control in comparisons:
    experimental_design['comparisons'].append({
        'case': case,
        'control': control,
        'name': '%s/%s' % (case, control)
    })
    

In [51]:
ds = DataSource(intdf, annodf, experimental_design, DATABASE_PIMP_KEGG)

2021-08-31 13:11:17.009 | DEBUG    | pals.feature_extraction:__init__:46 - Using PiMP_KEGG as database
2021-08-31 13:11:17.010 | DEBUG    | pals.loader:load_data:42 - Loading C:\Users\Simon\anaconda3\lib\site-packages\pals\data\PiMP_KEGG.json.zip
2021-08-31 13:11:17.050 | DEBUG    | pals.feature_extraction:__init__:59 - Mapping pathway to unique ids
2021-08-31 13:11:17.060 | DEBUG    | pals.feature_extraction:__init__:73 - Creating dataset to pathway mapping
2021-08-31 13:11:18.185 | DEBUG    | pals.feature_extraction:__init__:101 - Computing unique id counts


In [52]:
#Method for calculating p values from intensity matrix

def calculate_p(intensity_df):
    #students t for generating p values per compound

    comp = experimental_design['comparisons'][0]

    control_groups = ds.get_comparison_samples(comp)[0]
    case_groups = ds.get_comparison_samples(comp)[1]
    
    df = pd.DataFrame(columns = ['case', 'control']) #create case control df

    for sample_group in control_groups:
        df['control'] = intdf[sample_group] #populate with control data

    for sample_group in case_groups:
        df['case'] = intdf[sample_group] #populate with case data

    pvals = []

    df['pvalue'] = 0 #initialise p value column to 0

    for i in range(len(df.index)): #iterate through rows in dataframe calculating p value for each comparison
        
        case_log = np.log2(df['case'].iloc[i])
        control_log = np.log2(df['control'].iloc[i])

        if str(case_log) == 'nan': #filter nan values
            case_log = 1
        if str(control_log) == 'nan':
            control_log = 1

        statistics, pvalue = stats.ttest_ind(case_log, control_log) #do t test on the logged 2 values
        pvals.append(pvalue) # append to pval list 


    print(pvals) #debug list

In [45]:
calculate_p(intdf)

C:\Users\Simon\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Simon\anaconda3\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,